# Jupyter notebook based on ImageD11 to process scanning 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 16/01/2024

In [ ]:
# There is a bug with the current version of ImageD11 in the site-wide Jupyter env.
# This has been fixed here: https://github.com/FABLE-3DXRD/ImageD11/commit/4af88b886b1775585e868f2339a0eb975401468f
# Until a new release has been made and added to the env, we need to get the latest version of ImageD11 from GitHub
# Put it in your home directory somewhere
# USER: Change the path below to point to your local copy of ImageD11:

id11_code_path = "/home/esrf/james1997a/Code/ImageD11"

import sys

sys.path.insert(0, id11_code_path)

In [ ]:
# import functions we need

import os, glob, pprint

import ImageD11.sinograms.dataset
import ImageD11.sinograms.lima_segmenter
import ImageD11.sinograms.assemble_label
import ImageD11.sinograms.properties

In [ ]:
# Check that we're importing ImageD11 from the home directory rather than from the Jupyter kernel

?ImageD11.sinograms.dataset

In [ ]:
# USER: specify your experimental directory

base_dir = "/home/esrf/james1997a/Data/ihma439/id11/20231211"

rawdata_path = os.path.join(base_dir, 'RAW_DATA')

!ls -lrt {rawdata_path}

In [ ]:
# USER: pick a sample and a dataset you want to segment

sample = "FeAu_0p5_tR_nscope"
dataset = "top_700um"

# USER: specify path to detector mask

mask_path = '/data/id11/nanoscope/Eiger/eiger_mask_E-08-0173_20231127.edf'

processed_data_root_dir = os.path.join(base_dir, 'PROCESSED_DATA/James')  # USER: modify this to change the destination folder if desired
sparse_pixels_dir = os.path.join(processed_data_root_dir, "SparsePixels_NewMask")  # USER: modify this to change the name of the SparsePixels folder inside processed_data_root_dir

# makes folders if they don't already exist

if not os.path.exists(processed_data_root_dir):
    os.makedir(processed_data_root_dir)
    
if not os.path.exists(sparse_pixels_dir):
    os.makedirs(sparse_pixels_dir)

# desination of H5 files

dset_path = os.path.join(sparse_pixels_dir, f"ds_{sample}_{dataset}.h5" )
sparse_path = os.path.join(sparse_pixels_dir, f'{sample}_{dataset}_sparse.h5')
pks_path = os.path.join(sparse_pixels_dir, f'pks_{sample}_{dataset}.h5')

# create ImageD11 dataset object

ds = ImageD11.sinograms.dataset.DataSet(dataroot=rawdata_path,
                                        analysisroot=sparse_pixels_dir,
                                        sample=sample,
                                        dset=dataset)
ds.import_all()
ds.save(dset_path)

# create batch file to send to SLURM cluster

sbat = ImageD11.sinograms.lima_segmenter.setup(dset_path, maskfile=mask_path)
sbat

In [ ]:
# send batch file to SLURM cluster

!sbatch {sbat}

In [ ]:
# check status of SLURM jobs

!squeue -u $USER

In [ ]:
# you must wait until the SLURM jobs have completed before running the next cell!

In [ ]:
# label sparse peaks

ImageD11.sinograms.assemble_label.main(dset_path, sparse_path)

In [ ]:
# generate peaks table

ImageD11.sinograms.properties.main(dset_path, sparse_path, pks_path, options={'algorithm': 'lmlabel', 'wtmax': 70000, 'save_overlaps': False})